# 01_schema_inspection

Purpose:
- Explore raw CSV files downloaded by the ingestion pipeline.
- Validate schema, column types, missing values.
- Identify primary keys and foreign keys.

Dependencies :
- pandas
- pathlib

## Load raw files

In [3]:
import pandas as pd
from pathlib import Path

raw_path = Path("../data/raw/olist")

files = list(raw_path.glob("*.csv"))
files

[WindowsPath('../data/raw/olist/olist_customers_dataset.csv'),
 WindowsPath('../data/raw/olist/olist_geolocation_dataset.csv'),
 WindowsPath('../data/raw/olist/olist_orders_dataset.csv'),
 WindowsPath('../data/raw/olist/olist_order_items_dataset.csv'),
 WindowsPath('../data/raw/olist/olist_order_payments_dataset.csv'),
 WindowsPath('../data/raw/olist/olist_order_reviews_dataset.csv'),
 WindowsPath('../data/raw/olist/olist_products_dataset.csv'),
 WindowsPath('../data/raw/olist/olist_sellers_dataset.csv'),
 WindowsPath('../data/raw/olist/product_category_name_translation.csv')]

In [7]:
customers = pd.read_csv(raw_path / "olist_customers_dataset.csv")
orders = pd.read_csv(raw_path / "olist_orders_dataset.csv")
order_items = pd.read_csv(raw_path / "olist_order_items_dataset.csv")
payments = pd.read_csv(raw_path / "olist_order_payments_dataset.csv")
reviews = pd.read_csv(raw_path / "olist_order_reviews_dataset.csv")
products = pd.read_csv(raw_path / "olist_products_dataset.csv")
sellers = pd.read_csv(raw_path / "olist_sellers_dataset.csv")
geolocation = pd.read_csv(raw_path / "olist_geolocation_dataset.csv")
category_translation = pd.read_csv(raw_path / "product_category_name_translation.csv")

# Schema Inspection

## customers

In [25]:
customers.head()
#Everything looks in order

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [9]:
customers.info()
#Doesn't seem to contain missing values, Dtypes are correct

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [27]:
customers.describe(include='all')
# all customer_id are unique, most active customer placed 17 orders

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
count,99441,99441,99441.000000,99441,99441
unique,99441,96096,NaN,4119,27
top,06b8999e2fba1a1fbc88172c00ba8bc7,8d50f5eadf50201ccdcedfb9e2ac8455,NaN,sao paulo,SP
freq,1,17,NaN,15540,41746
mean,NaN,NaN,35137.474583,NaN,NaN
std,NaN,NaN,29797.938996,NaN,NaN
min,NaN,NaN,1003.000000,NaN,NaN
25%,NaN,NaN,11347.000000,NaN,NaN
50%,NaN,NaN,24416.000000,NaN,NaN
75%,NaN,NaN,58900.000000,NaN,NaN


Customers Table — Schema Inspection Summary
Structure

Rows: 99,441
Columns: 5
No missing values across all columns

Data types:

customer_id — object (string)
customer_unique_id — object (string)
customer_zip_code_prefix — int64
customer_city — object
customer_state — object

Key Notes
1. customer_id vs customer_unique_id
   
customer_id appears exactly once per row (freq = 1).
customer_unique_id has 96,096 unique values, meaning:
Some customers appear multiple times because they made multiple orders.
Most repeated customer appears 17 times → placed 17 separate orders.

Interpretation:
customer_id = order-level token (unique per order/session).
customer_unique_id = person-level identifier (stable across orders).
This is expected behavior in the Olist dataset and common in anonymized e-commerce data.

2. Zip code prefix

Numeric and clean.
No leading zeros in Brazilian postal codes → int64 is acceptable.
Represents only the prefix, not the full ZIP code.

3. City and State

Both clean and non-null.
customer_state has 27 unique values, matching Brazilian federative units (UFs).
customer_city has 4,119 unique values, indicating strong geographic diversity.

4. General Observations

No immediate data quality issues.
No duplicates to clean at this stage.
This table is order-level customer info, not a deduplicated customer registry.
Deduplication using customer_unique_id may be needed later for customer-level analytics (LTV, retention, segmentation).

5. Implications for the Cleaning Pipeline

Convert date fields later (none in this table).
IDs should remain as strings.
No missing values → no imputation required here.

Will join to:
orders on customer_id

## orders

Orders Table — Schema Inspection Summary
Structure

Rows: 99,441
Columns: 8
Data types: all columns are object, even timestamps

Missing values exist in several timestamp fields
order_id and customer_id are both unique (1:1 relationship with this table)

Column-by-Column Notes

1. order_id

99,441 unique → primary key.
Correct as string (object), because it is a hashed identifier.

2. customer_id

99,441 unique → each order links to exactly one customer entry.
This confirms customers table has 1 row per order (order-level customer info).
Will be used to join orders → customers.

3. order_status

8 unique values (e.g., “delivered”, “shipped”, “canceled”, etc.)
“delivered” is by far the most frequent (~96k).
Represents the final state of the order.
Should remain a categorical/dimension field.

4. Timestamps

All timestamp-related fields are incorrectly stored as strings (object):
order_purchase_timestamp
order_approved_at
order_delivered_carrier_date
order_delivered_customer_date
order_estimated_delivery_date

This is expected — Kaggle CSVs do not preserve datetime types.
They must be converted to proper datetime64 during cleaning.

5. Missing Values — Very Important

Several of the timestamp fields contain nulls:
Column	Missing?	Why

order_approved_at	Yes (~160)	Orders that were never approved or canceled early
order_delivered_carrier_date	Yes (~1,700)	Shipped late or never shipped
order_delivered_customer_date	Yes (~3,000)	Order not delivered yet or canceled
order_estimated_delivery_date	No	Always filled

This has direct implications:

Not all orders were completed (some canceled or not shipped).
Delivery performance analysis must handle missing timestamps.
Duration calculations might require filtering “delivered” status.

6. Duplicate timestamps

.describe() shows frequencies > 1 for some timestamps (e.g., 9 occurrences of a specific approved date).
This is normal — timestamps represent real-life events that can (and should) repeat.

No issue here.

7. Business Interpretation

The table describes the lifecycle of an order, including:

Purchase
Approval
Shipment
Delivery
Estimated delivery

This is the central table for:

Delivery performance
Order lead time
Cancellation analysis
Funnel analysis (purchase → delivered)

8. Implications for the Cleaning Pipeline

Convert all timestamp columns to datetime64.
Handle null timestamps appropriately, depending on status:
e.g., missing delivery dates for canceled orders.

Possibly create new features:

delivery_time = delivered_customer - purchase_timestamp
shipping_time = delivered_carrier - approval_timestamp
delay = delivered_customer - estimated_delivery_date
Normalize order_status (lowercase, categorical).

✔ Summary

The orders table is clean structurally but needs:

datetime conversion
missing-value handling
business-rule-aware filtering
This table will drive most of the analysis.

In [34]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [36]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [37]:
orders.describe(include="all")

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
count,99441,99441,99441,99441,99281,97658,96476,99441
unique,99441,99441,8,98875,90733,81018,95664,459
top,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2018-04-11 10:48:14,2018-02-27 04:31:10,2018-05-09 15:48:00,2018-05-08 23:38:46,2017-12-20 00:00:00
freq,1,1,96478,3,9,47,3,522


## order_items

In [38]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [39]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [40]:
order_items.describe(include="all")

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
count,112650,112650.000000,112650,112650,112650,112650.000000,112650.000000
unique,98666,NaN,32951,3095,93318,NaN,NaN
top,8272b63d03f5f79c56e9e4120aec44ef,NaN,aca2eb7d00ea1a7b8ebd4e68314663af,6560211a19b47992c3666cc44a7e94c0,2017-07-21 18:25:23,NaN,NaN
freq,21,NaN,527,2033,21,NaN,NaN
mean,NaN,1.197834,NaN,NaN,NaN,120.653739,19.990320
std,NaN,0.705124,NaN,NaN,NaN,183.633928,15.806405
min,NaN,1.000000,NaN,NaN,NaN,0.850000,0.000000
25%,NaN,1.000000,NaN,NaN,NaN,39.900000,13.080000
50%,NaN,1.000000,NaN,NaN,NaN,74.990000,16.260000
75%,NaN,1.000000,NaN,NaN,NaN,134.900000,21.150000


Structure

Rows: 112,650
Columns: 7
No missing values

Mixed types:

Identifiers → object
order_item_id → int64
shipping_limit_date → object (should be datetime)
price, freight_value → float64
This table represents line items within each order. An order can contain multiple products, so this table is larger than the orders table.

Primary Key

There is no single-column primary key.
Instead, it uses a composite key:
(order_id, order_item_id)

Why?
Because each order can contain multiple items (order_item_id = 1, 2, 3…).
This is normal in any e-commerce transaction database.

Key Relationships

order_id → links to the orders table
product_id → links to the products table
seller_id → links to the sellers table
This table will be critical for combining order-level and product-level information.

Important Columns

1. order_id

98,666 unique values (less than orders table)

Maximum frequency for a single order is 21 items
→ One customer bought 21 separate products in a single order.

2. order_item_id

Always starts at 1 for each order
Max value is 21, confirming the largest order had 21 items
Defines the ordering of items within an order

3. shipping_limit_date

String (object) type
Must be converted to datetime64 during cleaning
Represents the deadline for the seller to ship the product
(Important for SLA compliance metrics)

4. price

Product price for that item
Varies widely (min 0.85, max 6,735)

5. freight_value

Shipping cost charged to the customer
Has some zeros → these may represent:
free shipping

marketplace promotions
items shipped with other items under the same freight fee

Business Interpretation
This table allows analysis of:

Items per order
Revenue per product
Shipping fees vs product price
Seller performance
Contribution margins (optional)
Product mix inside each order

This table, combined with orders, enables complete GMV (gross merchandise value) calculation.

Implications for the Cleaning Pipeline

Convert shipping_limit_date to datetime
Ensure numeric columns (price, freight_value) stay float
Validate that (order_id, order_item_id) uniqueness holds

Potentially compute useful features like:

total_items_per_order
total_order_value
total_freight_per_order
This table becomes essential to build the aggregated order dataset later.

## payments

In [41]:
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [42]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [44]:
payments.describe(include="all")

,order_id,payment_sequential,payment_type,payment_installments,payment_value
count,103886,103886.000000,103886,103886.000000,103886.000000
unique,99440,NaN,5,NaN,NaN
top,fa65dad1b0e818e3ccc5cb0e39231352,NaN,credit_card,NaN,NaN
freq,29,NaN,76795,NaN,NaN
mean,NaN,1.092679,NaN,2.853349,154.100380
std,NaN,0.706584,NaN,2.687051,217.494064
min,NaN,1.000000,NaN,0.000000,0.000000
25%,NaN,1.000000,NaN,1.000000,56.790000
50%,NaN,1.000000,NaN,1.000000,100.000000
75%,NaN,1.000000,NaN,4.000000,171.837500


Payments Table — Schema Inspection Summary
Structure

Rows: 103,886
Columns: 5
No missing values

Data types:

order_id — object (string)
payment_sequential — int64
payment_type — object
payment_installments — int64
payment_value — float64

Payments represent how each order was paid, and some orders have multiple payments (e.g., split transactions).

Primary Key:
There is no primary key in this table.

Important observation:

order_id appears 99,440 times, matching the orders table.
But the payments table has 103,886 rows, meaning:
Some orders have more than one payment row.

This can represent:

multiple transactions for the same order,
partial payments,
refunds (rare),
installment transactions separated into multiple records.

Key Columns
1. order_id

Links to the orders table.
The top order ID appears 29 times, meaning the most extreme case had 29 separate payment records.
This matters in aggregation later.

2. payment_sequential

Starts at 1 for each order.
Max is 29, confirming the highest number of payment transactions for a single order.
Combined with order_id, it could form a composite key, but there is no strict guarantee.

Example:

(order_id, payment_sequential)

3. payment_type

5 unique categories.
Most common:

credit_card (≈ 74% of payments)

Other types usually include:
boleto
voucher
debit_card
not_defined

Payment type can be used for:

customer segmentation
logistic correlations (credit card orders ship faster because approval is immediate)

4. payment_installments

Number of installments chosen by the customer.
Ranges from 0 to 24.
Installments are extremely common in Brazil (consumer financing culture).
Installments are NOT split into separate rows; this is metadata.

5. payment_value

Monetary amount of that payment record.
Some values are 0.00, which may represent:
payment adjustments
voucher use
split-payment mechanics
Max payment value is unusually high (13,664), but plausible for multi-item orders.
Business Interpretation

This table allows:

Total revenue per order (sum(payment_value))
Payment method distribution
Analysis of installment preferences
Identifying unusual payments (multiple transactions, 0-value entries)
It is essential for calculating GMV (Gross Merchandise Value).

Implications for the Cleaning Pipeline

Validate that monetary columns remain float.
Confirm payments aggregate correctly per order.

Compute:

total_payment_value_per_order
number_of_payments_per_order

Ensure joins preserve multiplicity (one-to-many relationship with orders).

When creating the final cleaned dataset, I will almost certainly aggregate payments to a single row per order unless my analysis requires transaction-level detail.

Summary

The payments table is clean, complete, and well-structured.
It introduces a one-to-many relationship between orders and payments, and must be aggregated carefully during modeling.

## reviews

In [45]:
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [46]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [48]:
reviews.describe(include="all")

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
count,99224,99224,99224.000000,11568,40977,99224,99224
unique,98410,98673,NaN,4527,36159,636,98248
top,7b606b0d57b078384f0b58eac1d41d78,c88b1d1b157a9999ce368f218a407141,NaN,Recomendo,Muito bom,2017-12-19 00:00:00,2017-06-15 23:21:05
freq,3,3,NaN,423,230,463,4
mean,NaN,NaN,4.086421,NaN,NaN,NaN,NaN
std,NaN,NaN,1.347579,NaN,NaN,NaN,NaN
min,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,4.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,5.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,5.000000,NaN,NaN,NaN,NaN


Reviews Table — Schema Inspection Summary
Structure

Rows: 99,224
Columns: 7

Mixed data quality:
No missing values in IDs, scores, or timestamps
Heavy missingness in text fields

Data types:

review_id — object
order_id — object
review_score — int64
review_comment_title — object (many nulls)
review_comment_message — object (many nulls)
review_creation_date — object (should be datetime)
review_answer_timestamp — object (should be datetime)

Primary Key

review_id is intended to be the primary key, but:

unique(review_id) = 98,410
Rows = 99,224

So 814 review_id values are duplicated (mainly duplicates from system behavior).

Maximum frequency = 3 duplicated instances.

Important:
This does NOT necessarily mean three reviews were left — it could reflect data duplication during ETL on Olist’s side.

For my project, I may need to deduplicate reviews.

Relationships

order_id links the review to an entry in the orders table.

But:

Only 98,673 unique order_ids exist in reviews
vs 99,441 orders in the orders table

Meaning:

Some orders never received a review, which is realistic.
Some orders have more than one review record, which may require deduplication.

Key Columns
1. review_score

Integer from 1 to 5
Mean = 4.08, indicating a skew toward positive sentiment

Distribution:
Median = 5
25% = 4
75% = 5
This dataset is well known for having many “5-star: delivered on time” reviews.

2. review_comment_title and review_comment_message

Extremely sparse:

review_comment_title has only ~11,568 non-null values
review_comment_message has ~40,977 non-null values

Meaning:

Most customers leave a numeric rating only, without text.
This limits text analysis but still allows sentiment exploration based on ratings.

3. Timestamps

review_creation_date and review_answer_timestamp are both strings
Must be converted to datetime
These columns allow you to compute:
response time (difference between creation and answer timestamp)
review churn (delay between delivery and review)

Business Interpretation

The reviews table measures:

Customer satisfaction
Seller responsiveness
Service quality and delivery reliability
Product experience

This is a key table for customer experience KPIs.

Examples of useful metrics:

Average review score by category/seller
Review score distribution
Review response times
Correlation between delays and low scores
Percentage of orders receiving a review
Data Quality Observations

Text fields are sparse → handle with caution
Duplicate reviews exist → deduplication or aggregation needed
Some orders have multiple reviews → decide how to handle (e.g., keep the earliest, or the highest priority record)

Implications for the Cleaning Pipeline

Convert timestamp fields to datetime
Deduplicate based on review_id or (order_id, review_creation_date)
Standardize text fields (optional, depending on your analysis depth)

Create features:

review_response_time
has_review_comment (boolean)

This table will enrich my final analytical dataset with sentiment and service quality metrics.

## products

In [49]:
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [50]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [51]:
products.describe(include="all")

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32951,32341,32341.000000,32341.000000,32341.000000,32949.000000,32949.000000,32949.000000,32949.000000
unique,32951,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,1e9e8ef04dbcff4541ed26657ea517e5,cama_mesa_banho,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,3029,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,48.476949,771.495285,2.188986,2276.472488,30.815078,16.937661,23.196728
std,NaN,NaN,10.245741,635.115225,1.736766,4282.038731,16.914458,13.637554,12.079047
min,NaN,NaN,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,NaN,NaN,42.000000,339.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,NaN,NaN,51.000000,595.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,NaN,NaN,57.000000,972.000000,3.000000,1900.000000,38.000000,21.000000,30.000000


Products Table — Schema Inspection Summary
Structure

Rows: 32,951
Columns: 9

Mixed data quality:

Missing values in several fields
Possible outliers in weight and dimensions

Data types:

7 numeric columns (float64)
2 categorical columns (product_id, product_category_name)

Primary Key

product_id is unique → primary key.
This is a clean, well-defined identifier.

Missing Values in:

product_category_name → ~610 missing
product_name_lenght → ~610 missing
product_description_lenght → ~610 missing
product_photos_qty → ~610 missing
product_weight_g → 2 missing
product_length_cm → 2 missing
product_height_cm → 2 missing
product_width_cm → 2 missing

These missing 610 values are clustered together, meaning:

Some products in Olist do not have any metadata (no category, no text length, no photos).
This is a known quirk of the dataset.

I must decide whether to:

drop these products,
impute them,
or keep them only if they appear in order_items.

Column Notes
1. product_category_name

73 unique categories.

Names are in Portuguese (e.g., perfumaria, esporte_lazer, cama_mesa_banho).
A translation table exists (product_category_name_translation.csv).
I want join this table later to provide English names.

2. product_name_lenght, product_description_lenght

These represent:

length of the product name
length of the product description

Useful for:

detecting extremely short/invalid listings
modeling completeness of product metadata
Missing ~610 values → incomplete listings.

3. product_photos_qty

Typically 1–3 photos
Max = 20 (rare, but plausible)
Missing ~610 values
This field can indicate listing quality.

4. product dimensions & weight

Columns:

product_weight_g
product_length_cm
product_height_cm
product_width_cm

Observations:

Some values are extremely small (weight = 0, height = 2 cm)
Some values very large (weight = 40 kg, dimensions > 1 meter)
This is realistic for Olist (from small accessories to furniture)

But:

A weight of 0g cannot be real
2cm height might be packaging error
We should flag extreme values in cleaning.

Business Interpretation
The products table represents:

product metadata
physical dimensions (used for freight calculation)
quality indicators (name length, description length, photos)

category information

This table is central for logistics analysis and product-level insights.

Data Quality Observations

Many products lack metadata (category, name length, etc.)
Dimensions and weight include unrealistic values (0g, 2cm)
Text-length fields are floats (should be integers)
Categories are untranslated

I will address these issues in the cleaning pipeline.

Implications for the Cleaning Pipeline

Convert text-length columns to integers
Convert weight/dimension columns to integers
Fix or drop rows with invalid weight/dimensions
Handle missing metadata appropriately (strategy needed)
Left join with the translation table for English category names

This table is critical for:

freight cost modelling
category-level analysis
item-level profitability and performance metrics

Summary

The products table is structurally simple but contains notable quality issues that must be cleaned before analysis. Missing metadata and unrealistic dimensions require careful handling.

## sellers

In [52]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [53]:
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


In [55]:
sellers.describe(include="all")

,seller_id,seller_zip_code_prefix,seller_city,seller_state
count,3095,3095.000000,3095,3095
unique,3095,NaN,611,23
top,3442f8959a84dea7ee197c632cb2df15,NaN,sao paulo,SP
freq,1,NaN,694,1849
mean,NaN,32291.059451,NaN,NaN
std,NaN,32713.453830,NaN,NaN
min,NaN,1001.000000,NaN,NaN
25%,NaN,7093.500000,NaN,NaN
50%,NaN,14940.000000,NaN,NaN
75%,NaN,64552.500000,NaN,NaN


Sellers Table — Schema Inspection Summary
Structure

Rows: 3,095
Columns: 4

No missing values

Data types:

seller_id — object
seller_zip_code_prefix — int64
seller_city — object
seller_state — object

This table is clean and simple — it serves as a lookup table for seller metadata.

Primary Key

seller_id is unique → primary key.

Column Notes
1. seller_id

3,095 unique sellers.
Each seller appears exactly once.
Clean and ready for joining.

2. seller_zip_code_prefix

Numeric postal prefix (not a full ZIP code).
Values range from 1001 to ~99,730.
This mirrors customer ZIP code structure.
No missing values.
You do not convert this to string because Brazilian postal prefixes don’t have leading zeros.

3. seller_city and seller_state

seller_city has 611 unique cities.
seller_state has 23 states, not all 27 Brazilian UFs.
This is expected — not every state has sellers.
Top state is SP (São Paulo) with 1,849 sellers — this matches Brazil’s real economic distribution.

Business Interpretation

The sellers table is essential for:

Geographic analysis of supply
Seller concentration
Delivery performance by region
Freight cost variations
Linking to order_items via seller_id
Sellers are the supply side of the marketplace; understanding their distribution is critical.

Data Quality Observations

Table is clean (no missing values, no duplicates).
No unrealistic values.

No further cleaning required beyond:

lowercase normalization (optional)
consistent joining

This table is significantly cleaner than products and reviews.

Implications for the Cleaning Pipeline

Minimal cleaning required:

Ensure city/state text normalization (lowercasing if needed).
Keep ZIP code as integer.
Validate seller_id joins with order_items.

This table will join with order_items to:
compute seller-level metrics
calculate revenue per seller
analyze freight patterns
map geographic supply distribution

Summary

The sellers table is clean, complete, and structurally simple.
It will play a key role once joined with order_items for marketplace-level analysis.

## geolocation

In [56]:
geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [57]:
geolocation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


In [58]:
geolocation.describe(include="all")

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
count,1.000163e+06,1.000163e+06,1.000163e+06,1000163,1000163
unique,NaN,NaN,NaN,8011,27
top,NaN,NaN,NaN,sao paulo,SP
freq,NaN,NaN,NaN,135800,404268
mean,3.657417e+04,-2.117615e+01,-4.639054e+01,NaN,NaN
std,3.054934e+04,5.715866e+00,4.269748e+00,NaN,NaN
min,1.001000e+03,-3.660537e+01,-1.014668e+02,NaN,NaN
25%,1.107500e+04,-2.360355e+01,-4.857317e+01,NaN,NaN
50%,2.653000e+04,-2.291938e+01,-4.663788e+01,NaN,NaN
75%,6.350400e+04,-1.997962e+01,-4.376771e+01,NaN,NaN


Geolocation Table — Schema Inspection Summary
Structure

Rows: 1,000,163
Columns: 5
No missing values

Data types:

geolocation_zip_code_prefix — int64
geolocation_lat, geolocation_lng — float64
geolocation_city, geolocation_state — object

This is by far the largest table in the dataset.

Critical Note: This Table Is NOT a Lookup Table
It's easy to assume this table maps each ZIP prefix to a single latitude/longitude.
This is incorrect.

In reality:

Each ZIP code prefix appears many times (hundreds or thousands of rows).
Each row represents a GPS coordinate where a user activity occurred.
This includes customers AND sellers, AND general anonymized geographic samples.

This is why:

geolocation_city has 8,011 unique cities
geolocation_state includes all 27 Brazilian UFs
São Paulo appears 135,800+ times

It is not a clean mapping table and is not used for direct joining to orders or customers.

Why does this table exist?

The geolocation table is intended to:

provide a dense mapping of lat/lng coordinates to postal prefixes
support geographic clustering
support distance calculations

But it requires preprocessing because:

ZIP prefixes do not uniquely map to a single latitude/longitude.

Example:
geolocation_zip_code_prefix = 1046 appears multiple times, with different latitude/longitude coordinates.

This is expected — ZIP prefixes cover regions, not points.

Business Interpretation

This table supports:

approximating customer locations
approximating seller locations
distance computation:
customer → seller
customer → warehouse (if applicable)
seller → distribution center

However:

You must aggregate or filter the table before use.
For example, many analysts take the average lat/lng per ZIP prefix.

Data Quality Observations

No missing values → structurally complete
Extreme lat/lng values exist but are bounded and valid
ZIP prefixes range from ~1000 to ~99990
Repetition is intentional

Implications for the Cleaning Pipeline

I cannot join this table directly to customers or sellers without processing.

Recommended cleaning steps:

1. Aggregate lat/lng per ZIP prefix

Compute:

geo_grouped = geolocation.groupby("geolocation_zip_code_prefix").agg({
    "geolocation_lat": "mean",
    "geolocation_lng": "mean"
})


This gives a single representative coordinate per prefix.

2. Ensure city/state consistency (optional)

Because city/state can differ across rows of the same ZIP prefix, we could pick :

the most frequent city
the most frequent state

3. Then keep only the final aggregated mapping

Save it as:
data/processed/geolocation_cleaned.csv

4. Join with customers and sellers

Using their ZIP prefixes.

Key Warning

I should never use the raw geolocation table directly in joins or distance calculations.
Instead I should always use the aggregated version.

Summary

The geolocation table is large, complete, and intentionally dense.
It must be aggregated before use and plays a crucial role in geographic and logistics analysis.

## category_translation

In [59]:
category_translation.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [60]:
category_translation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [61]:
category_translation.describe(include="all")

,product_category_name,product_category_name_english
count,71,71
unique,71,71
top,beleza_saude,health_beauty
freq,1,1


Category Translation Table — Schema Inspection Summary
Structure

Rows: 71
Columns: 2
No missing values
Both columns are strings (object)
Clean and simple mapping table
This table provides English translations for Portuguese product category names.

Primary Key

product_category_name is unique → primary key
Every category has exactly one English equivalent.

Contents

71 Portuguese category names
71 English translations
All values are unique

Examples:

Portuguese	English
beleza_saude	health_beauty
cama_mesa_banho	bed_bath_table
moveis_decoracao	furniture_decor
informatica_acessorios	computers_accessories

The English names are clean, standardized, and lowercase with underscores.

Business Interpretation

The products table uses Portuguese category names:
perfumaria
esporte_lazer
bebes
utilidades_domesticas

…

To make the dataset usable internationally it's mandatory that I translate categories.

This table enables:

category-level analysis in English
more readable dashboards
better interpretation of revenue by category

Data Quality Observations

Perfectly clean
No duplicates
No missing values
Two columns only
Entire table is ready for merging

Implications for the Cleaning Pipeline

During product cleaning:

1. Left join on product_category_name

This adds the English translation:

products_clean = products.merge(
    category_translation,
    on="product_category_name",
    how="left"
)

2. Missing category translations

The products table has ~610 missing category values → these will produce missing English names too.
You will need to decide how to handle them later (drop, impute, or “unknown”).

3. Use the English name in dashboards

In Power BI:

Use product_category_name_english as the primary category field
Optionally hide the Portuguese version
This significantly improves readability.

Summary

A small, clean, complete lookup table used to translate product categories into English.
Essential for clear reporting and analysis.

# Relational Schema

Olist Relational Schema Map
Main Tables

These represent the core business entities:

orders
PK: order_id
FKs:
customer_id → customers.customer_id

Relationships:

1 order → several order_items
1 order → several payments
1 order → 0 or 1 review

customers

PK: customer_id
Business key: customer_unique_id

Relationships:

1 customer → multiple orders

order_items
PK: (order_id, order_item_id)
FKs:

order_id → orders.order_id
product_id → products.product_id
seller_id → sellers.seller_id

Relationship:

Many-to-1 with orders
Many-to-1 with products
Many-to-1 with sellers

payments

No PK
FK: order_id → orders.order_id

Relationship:

1 order → multiple payments

During cleaning, payments must be aggregated to 1 row per order.

reviews

PK: imperfect review_id (duplicates exist)
FK: order_id → orders.order_id

Relationship:

1 order → 0 or 1 review (theoretically)

Some orders have multiple review rows
→ deduplication required during cleaning.

products

PK: product_id
FKs: none

Relationship:

1 product → many order_items

sellers

PK: seller_id

Relationship:

1 seller → many order_items

geolocation

No PK
Many rows per ZIP prefix

Must be aggregated during cleaning:

group by geolocation_zip_code_prefix, take average lat/lng

FKs:

join to customers and sellers on ZIP prefix (after aggregation)

category_translation

PK: product_category_name

Used to translate categories in the products table.


Relational Schema :

<pre>
customers (1) ────< (N) orders (1) ────< (N) order_items (N) >──── products
                                 │
                                 ├───< (N) payments (aggregated)
                                 │
                                 └───< (N) reviews (deduplicated)

order_items (N) >──── sellers (1)

customers ─── ZIP prefix ───> geolocation (aggregated)
sellers   ─── ZIP prefix ───> geolocation (aggregated)

products ─── product_category_name ───> category_translation
</pre>

